In [1]:
import pyAgrum as gum

# Red Bayesiana

In [2]:
# Create Bayes Net
bn = gum.BayesNet()

In [3]:
# Add variables

In [4]:
b = bn.add(gum.LabelizedVariable('b', 'Burglary ?', 2))
print(b)

0


In [5]:
e = bn.add(gum.LabelizedVariable('e', 'Earthquake ?', 2))
print(e)

1


In [6]:
m = bn.add(gum.LabelizedVariable('m', 'Mary_calls ?', 2))
print(m)

2


In [7]:
j = bn.add(gum.LabelizedVariable('j', 'John_calls ?', 2))
print(j)

3


In [8]:
a = bn.add(gum.LabelizedVariable('a', 'Alarm ?', 2))
print(a)

4


In [9]:
# Add arcs

In [10]:
bn.addArc(b, a)
bn.addArc(e, a)
bn.addArc(a, m)
bn.addArc(a, j)

In [11]:
print(bn)

BN{nodes: 5, arcs: 4, domainSize: 32, dim: 20}


In [12]:
import pyAgrum.lib.notebook as gnb

In [13]:
bn

(gum::BayesNet<double>@0x1b2fe10) BN{nodes: 5, arcs: 4, domainSize: 32, dim: 20}

In [14]:
# Add values

### B

In [15]:
bn.cpt(b).fillWith([0.999,0.001])

(gum::Potential<double>@0x1b22f10) 
  b                |
0        |1        |
---------|---------|
 0.9990  | 0.0010  |

### E

In [16]:
bn.cpt(e).fillWith([0.998,0.002])

(gum::Potential<double>@0x1764980) 
  e                |
0        |1        |
---------|---------|
 0.9980  | 0.0020  |

### A

In [17]:
bn.cpt(a)[{'b' : 1, 'e' : 1, 'a' : 1}] = 0.95
bn.cpt(a)[{'b' : 1, 'e' : 1, 'a' : 0}] = 0.05
bn.cpt(a)[{'b' : 1, 'e' : 0, 'a' : 1}] = 0.94
bn.cpt(a)[{'b' : 1, 'e' : 0, 'a' : 0}] = 0.06
bn.cpt(a)[{'b' : 0, 'e' : 1, 'a' : 1}] = 0.29
bn.cpt(a)[{'b' : 0, 'e' : 1, 'a' : 0}] = 0.71
bn.cpt(a)[{'b' : 0, 'e' : 0, 'a' : 1}] = 0.001
bn.cpt(a)[{'b' : 0, 'e' : 0, 'a' : 0}] = 0.999
bn.cpt(a)

(gum::Potential<double>@0x1b08d50) 
             ||  a                |
b     |e     ||0        |1        |
------|------||---------|---------|
0     |0     || 0.9990  | 0.0010  |
1     |0     || 0.0600  | 0.9400  |
0     |1     || 0.7100  | 0.2900  |
1     |1     || 0.0500  | 0.9500  |

### M

In [18]:
bn.cpt(m)[{'a' : 1, 'm' : 1}] = 0.7
bn.cpt(m)[{'a' : 1, 'm' : 0}] = 0.3
bn.cpt(m)[{'a' : 0, 'm' : 1}] = 0.01
bn.cpt(m)[{'a' : 0, 'm' : 0}] = 0.99
bn.cpt(m)

(gum::Potential<double>@0x17385f0) 
      ||  m                |
a     ||0        |1        |
------||---------|---------|
0     || 0.9900  | 0.0100  |
1     || 0.3000  | 0.7000  |

### J

In [19]:
bn.cpt(j)[{'a' : 1, 'j' : 1}] = 0.9
bn.cpt(j)[{'a' : 1, 'j' : 0}] = 0.1
bn.cpt(j)[{'a' : 0, 'j' : 1}] = 0.05
bn.cpt(j)[{'a' : 0, 'j' : 0}] = 0.95
bn.cpt(j)

(gum::Potential<double>@0x18c3d00) 
      ||  j                |
a     ||0        |1        |
------||---------|---------|
0     || 0.9500  | 0.0500  |
1     || 0.1000  | 0.9000  |

# Respuestas

In [20]:
inf1 = gum.LazyPropagation(bn)

## $P(+b,-e,+a,-j,+m)$

In [21]:
inf1.addJointTarget({b, e, a, j, m})
resg = inf1.jointPosterior({b, e, a, j, m})* ### $Resultado segunda forma$

* ### $Resultado: primera forma$

In [22]:
print((resg[0][1][1][0][1]*100), '%')

0.006566840000000001 %


## $P(+b,-e,+a,-j,+m) => P(-e) P(+b) P(+m|+a) P(-j|+a) P(+a|-e,+b)$

* ### $P(+a|-e,+b)$

In [23]:
print(resg[1][0][1][0][1])

4.05594e-05


In [24]:
inf = gum.LazyPropagation(bn)

In [25]:
inf.setEvidence({'e':0, 'b':1})
inf.makeInference()
a = inf.posterior(a)[1]
print(a)

0.94


* ### $P(-j|+a)$

In [26]:
inf.setEvidence({'a':1})
inf.makeInference()
j = inf.posterior(j)[0]
print(j)

0.1


* ### $P(+m|+a)$

In [27]:
inf.setEvidence({'a':1})
mm = inf.posterior(m)[1]
print(mm)

0.7


* ### $P(+b)$

In [28]:
b = bn.cpt(b)[1]
print(b)

0.001


* ### $P(-e)$

In [29]:
e = bn.cpt(e)[0]
print(e)

0.998


* ### $Resultado: segunda forma$

In [30]:
res = a * b * e * j * mm
print(a)
print(b)
print(e)
print(j)
print(mm)
print(res)
print(res*100,'%')

0.94
0.001
0.998
0.1
0.7
6.56684e-05
0.00656684 %


## $P(M)$

In [31]:
inf.makeInference()
inf.posterior(m)

(gum::Potential<double>@0x1e209d0) 
  m                |
0        |1        |
---------|---------|
 0.3000  | 0.7000  |

## $P(M|+b,+a)$

In [32]:
inf.setEvidence({'b':1, 'a':1})
inf.makeInference()
inf.posterior(m)

(gum::Potential<double>@0x14759a0) 
  m                |
0        |1        |
---------|---------|
 0.3000  | 0.7000  |